# MesoNet

## Config

In [8]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Imports

In [15]:
from datetime import datetime
import pathlib
import typing as t

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.applications import xception
from tensorflow.keras import preprocessing
from tensorflow.keras import metrics
from tensorflow.keras import callbacks
from tensorflow.keras import models

## Constants

In [10]:
DATASET_PATH = pathlib.Path("D:/datasets/celeb_df/")
MODELS_PATH = pathlib.Path("../../../saved_models")
# TensorBoard logs path
LOGS_PATH = pathlib.Path("./logs")
IMAGE_SIZE = (256, 256)
INPUT_SHAPE = (*IMAGE_SIZE, 3)

## Load datasets

In [11]:
SEED = int(datetime.today().timestamp())
VALIDATION_SPLIT = 0.05
BATCH_SIZE = 32

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
)

Found 2342158 files belonging to 2 classes.
Using 2225051 files for training.


In [12]:
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
)

Found 2342158 files belonging to 2 classes.
Using 117107 files for validation.


In [13]:
REALS_TO_FAKE_RATIO = (
    len(list(DATASET_PATH.joinpath("reals").iterdir()))
    / len(list(DATASET_PATH.joinpath("fakes").iterdir()))
)
REALS_TO_FAKE_RATIO

0.10647836701990959

## Define model

In [11]:
def build_meso_net() -> None:
    model = keras.Sequential()
    model.add(layers.InputLayer(INPUT_SHAPE))
    # First block
    model.add(layers.Conv2D(8, (3, 3), padding="same", activation = "relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(2, 2), padding="same"))
    # Second block
    model.add(layers.Conv2D(8, (5, 5), padding="same", activation = "relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(2, 2), padding="same"))
    # Third block
    model.add(layers.Conv2D(16, (5, 5), padding="same", activation = "relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(2, 2), padding="same"))
    # Fourth layer
    model.add(layers.Conv2D(16, (5, 5), padding="same", activation = "relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(4, 4), padding="same"))
    # Top
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(16))
    model.add(layers.LeakyReLU(alpha=0.1))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation="sigmoid"))
    
    return model

In [12]:
model = build_meso_net()

## Training

In [21]:
METRICS = [
        metrics.BinaryAccuracy(),
        metrics.AUC(),
        #metrics.Precision(),
        #metrics.Recall(),
        #metrics.TruePositives(),
        #metrics.TrueNegatives(),
        #metrics.FalsePositives(),
        #metrics.FalseNegatives(),
    ]

In [22]:
# Save each run in individual directory
log_dir = LOGS_PATH.joinpath("fit").joinpath(datetime.now().strftime("%Y%m%d-%H%M%S"))

CALLBACKS = [
    callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
    ),
    callbacks.ModelCheckpoint(
        filepath=MODELS_PATH.joinpath("meso_net_{epoch:02d}_{val_loss:.2f}.h5"),
        monitor='val_loss',
        save_best_only=True,
    ),
    callbacks.TensorBoard(log_dir=log_dir),
]

In [23]:
# Params following "FaceForensics++: Learning to Detect Manipulated Facial Images"
optimizer = optimizers.Adam(
    learning_rate=1e-3,
    epsilon=1e-08
)

In [24]:
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=METRICS
)

In [25]:
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    class_weight={
        # 0 are fakes since they are first in alphabetical ordering
        0: REALS_TO_FAKE_RATIO,
        1: 1.0,
    },
    epochs=10,
    callbacks=CALLBACKS
)

Epoch 1/10
41835/69533 [=================>............] - ETA: 25:27 - loss: 0.1050 - binary_accuracy: 0.7053 - auc_2: 0.7916 - precision_1: 0.2024 - recall_1: 0.7032 - true_positives_1: 90416.0000 - true_negatives_1: 853753.0000 - false_positives_1: 356396.0000 - false_negatives_1: 38155.0000- ETA: 53:23 - loss: 0.1330 - binary_accuracy: 0.5535 - auc_2: - ETA: 53:14 - loss: 0.1329 - binary_accuracy: 0.5536 - auc_2: 0.5625 - precision_1: 0.1133 - recall_1: 0.5312 - true_positives_1: 15884.0000 - true_negatives_1: 155602.0000  - ETA: 53:11 - loss: 0.1329 - binary_accuracy: 0.5535 - auc_2: 0.5631 - precision_1: 0.1134 - recall_1: 0.5319 - true_positives_1: 16039.00 - ETA: 53:06 - loss: 0.1328 - binary_accuracy: 0.5551 - auc_2: 0.5644 - precision_1: 0.1138 - recall_1: 0.5321 - true_positives_1: 16258.0000 - true_neg - ETA: 53:02 - loss: 0.1327 - binary_accuracy: 0.5560 - auc_2: 0.5654 - precision_1: 0.1140 - recall_1: 0.5325 - true_positives_1: 16448.0000 - true_negatives_1: 161690.0000 -

69533/69533 [==============================] - 4107s 59ms/step - loss: 0.0481 - binary_accuracy: 0.9054 - auc_2: 0.9635 - precision_1: 0.5047 - recall_1: 0.8958 - true_positives_1: 191670.0000 - true_negatives_1: 1822965.0000 - false_positives_1: 188125.0000 - false_negatives_1: 22291.0000 - val_loss: 0.1858 - val_binary_accuracy: 0.9374 - val_auc_2: 0.9831 - val_precision_1: 0.6204 - val_recall_1: 0.9249 - val_true_positives_1: 10571.0000 - val_true_negatives_1: 99210.0000 - val_false_positives_1: 6468.0000 - val_false_negatives_1: 858.0000 auc_2: 0.9635 - precision_1: 0.5046 - recall_1: 0.8957 - t
Epoch 6/10
69533/69533 [==============================] - 4101s 59ms/step - loss: 0.0464 - binary_accuracy: 0.9105 - auc_2: 0.9662 - precision_1: 0.5202 - recall_1: 0.8992 - true_positives_1: 192403.0000 - true_negatives_1: 1833617.0000 - false_positives_1: 177473.0000 - false_negatives_1: 21558.0000 - val_loss: 0.2916 - val_binary_accuracy: 0.9076 - val_auc_2: 0.9835 - val_precision_1: 0.5

In [26]:
history.history

{'loss': [0.0925951823592186,
  0.06220102682709694,
  0.05459916964173317,
  0.05043787509202957,
  0.04811658337712288,
  0.046391695737838745],
 'binary_accuracy': [0.7567076086997986,
  0.86739182472229,
  0.889367938041687,
  0.8990535736083984,
  0.905433177947998,
  0.9105499386787415],
 'auc_2': [0.848120391368866,
  0.9384376406669617,
  0.9529570937156677,
  0.9599651098251343,
  0.963542103767395,
  0.9661611318588257],
 'precision_1': [0.2481044977903366,
  0.40980762243270874,
  0.4606282114982605,
  0.48643064498901367,
  0.5046669840812683,
  0.5201824307441711],
 'recall_1': [0.7535251975059509,
  0.8611148595809937,
  0.8803800940513611,
  0.8921672701835632,
  0.8958174586296082,
  0.8992432951927185],
 'true_positives_1': [161225.0,
  184245.0,
  188367.0,
  190889.0,
  191670.0,
  192403.0],
 'true_negatives_1': [1522488.0,
  1745746.0,
  1790522.0,
  1809551.0,
  1822965.0,
  1833617.0],
 'false_positives_1': [488602.0,
  265344.0,
  220568.0,
  201539.0,
  188125.

In [20]:
model_path = pathlib.Path.home().joinpath("Desktop/saved_models/meso_net_04_0.14.h5")

In [21]:
model = models.load_model(model_path)

In [23]:
model.evaluate(validation_ds)

3660/3660 [==============================] - 1023s 277ms/step - loss: 0.1415 - binary_accuracy: 0.9541 - auc_2: 0.9772 - precision_1: 0.7320 - recall_1: 0.8308 - true_positives_1: 9443.0000 - true_negatives_1: 102284.0000 - false_positives_1: 3457.0000 - false_negatives_1: 1923.0000


[0.14153720438480377,
 0.9540591239929199,
 0.9772054553031921,
 0.7320154905319214,
 0.8308112025260925,
 9443.0,
 102284.0,
 3457.0,
 1923.0]

In [14]:
%tensorboard --logdir logs/fit/20210813-084655